In [38]:
import regex as re

### Extracting Text from PDF

In [39]:
!pip install pdfminer.six
!pip install nltk spacy==2.3.5 https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
! python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")

import pdfminer
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
!pip install docx2txt
import docx2txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached spacy-2.3.5.tar.gz (5.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached thinc-7.4.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached srsly-1.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (369 kB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for spacy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for spacy
  Created

In [40]:
import os

In [41]:
rootdir = '/content/resumes'
files = os.listdir(rootdir)
file_names = []
for file in os.listdir(rootdir):
    file_names.append(os.path.join('resumes', file))
file_names

['resumes/Lakhani’s resume.pdf',
 'resumes/Apoorvgupta[9_0].pdf',
 'resumes/Arshiya’s resume-1.pdf',
 'resumes/Monika’s resume_220928_195956.pdf',
 'resumes/Harshil’s resume.pdf',
 'resumes/APZNZA~1.PDF',
 'resumes/Gursanjam’s resume-2.pdf',
 'resumes/Database management system (askbooks.net).pdf']

In [42]:
def open_pdf_file(file_name):
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    pagenums = set()
    infile = open(file_name, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()

    result = []

    for line in text.split('\n'):
        line2 = line.strip()
        if line2 != '':
            result.append(line2)
    return (result)

In [43]:
def open_docx_file(file_name):
    temp = docx2txt.process(file_names[0])
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    text = [t for t in text if len(t) > 1]
    return (text)

In [44]:
def remove_punctuations(line):
    return re.sub(r'(\.|\,)', '', line)

def preprocess_document(document):
    for index, line in enumerate(document):
        line = line.lower()
        line = remove_punctuations(line)

        line = line.split(' ')
        while '' in line:
            line.remove('')

        while ' '  in line:
            line.remove(' ')


        document[index] = ' '.join(line)
    return (document)



In [45]:
def get_email(document):
    #Further optimization to be done.
    emails = []
    pattern = re.compile(r'\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}')
    for line in document:
        matches = pattern.findall(line)
        for mat in matches:
            if len(mat) > 0:
                emails.append(mat)
    #print (emails)
    return (emails)


In [46]:
def get_phone_no(document):
    #This function has to be further modified better and accurate results.
    #Possible phone number formats - Including +91 or just with the numbers.

    mob_num_regex = r'''(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)
                        [-\.\s]*\d{3}[-\.\s]??\d{4}|\d{5}[-\.\s]??\d{4})'''
    pattern = re.compile(mob_num_regex)
    matches = []
    for line in document:
        match = pattern.findall(line)
        for mat in match:
            if len(mat) > 9:
                matches.append(mat)

    return (matches)



In [47]:
def get_education(document):
    education_terms = []
    with open('education.txt', 'r') as file:
        education_terms = file.readlines()

    education_terms = [term.strip('\n') for term in education_terms]
    education = []
    for line in document:
        for word in line.split(' '):
            if len(word) > 2 and word in education_terms:
                if line not in education:
                    education.append(line)
    #print (education)
    return (education)

In [48]:
def get_skills(document):
    skill_terms = []
    with open('valid_skills.txt', 'r') as file:
        skill_terms = file.readlines()

    skill_terms = [term.strip('\n') for term in skill_terms]
    skills = []

    for line in document:
        words = line.split(' ')

        for word in words:
            if word in skill_terms:
                if word not in skills:
                    skills.append(word)

        word_pairs = []
        for i in zip(words[:-1], words[1:]):
            word_pairs.append(i[0] + ' ' + i[1])   #This is to find skills like 'data science' i.e skills containint two words.    return (skills)

        for pair in word_pairs:
            if pair in skill_terms:
                if pair not in skills:
                    skills.append(pair)

    return (skills)

Common forms of experience in resume:
        
        Experience followed by the keyword 'experience'
        
        Projects followed by the keyword 'project'
        
        Dated of the form 'Jan 2019 - Feb 2019'
        
        Dates of the from '05/2019 to 06/2019'

In [49]:
def get_experience(document):
    pattern1 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4})')
    pattern2 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(\d{2}(.|..)\d{4})')
    pattern3 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(present)')
    pattern4 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(present)')
    patterns = [pattern1, pattern2, pattern3, pattern4]
    experience = []
    for index, line in enumerate(document):
        for pattern in patterns:
            exp = pattern.findall(line)
            if len(exp) > 0:
                experience.append(document[index:index+4])

    return (experience)


In [65]:
from spacy.matcher import Matcher
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(text):

  nlp_text = nlp(text)

  # First name and Last name are always Proper Nouns
  pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

  matcher.add('NAME', [pattern])

  matches = matcher(nlp_text)

  for match_id, start, end in matches:
      span = nlp_text[start:end]
      return span.text

In [66]:
names = []
email_ids = []
phone_nos = []
education_1 = []
education_2 = []
skills_1 = []
skills_2 = []
experience_1 = []
experience_2 = []
skills_ls = []

for file_name in file_names:
    if file_name.endswith('.pdf'):
        document = open_pdf_file(file_name)
    elif file_name.endswith('.docx'):
        document = open_docx_file(file_name)


    email = get_email(document)
    phone_no = get_phone_no(document)
    document = preprocess_document(document)
    name = extract_name(str(document))
    #print ('\n\n')
    #print (file_name)
    #print ('Email is {} phone number is {}'.format(email, phone_no))
    if len(email) > 0:
        email_ids.append(email[0])
    else:
        email_ids.append('')

    if len(phone_no) > 0:
        phone_nos.append(phone_no[0])
    else:
        phone_nos.append('')

    education = get_education(document)
    #print ('Education ', get_education(document))
    if len(education) > 1:
        education_1.append(education[0])
        education_2.append(education[1])
    elif len(education) == 1:
        education_1.append(education[0])
        education_2.append('')
    elif len(education) == 0:
        education_1.append('')
        education_2.append('')

    skills = get_skills(document)
    # print ('Skills ', skills)
    all_skills = ' '
    skills_ls.append(all_skills.join(skills))

    if len(skills) > 1:
        skills_1.append(skills[0])
        skills_2.append(skills[1])
    elif len(skills) == 1:
        skills_1.append(skills[0])
        skills_2.append('')
    elif len(skills) == 0:
        skills_1.append('')
        skills_2.append('')

    experience = get_experience(document)
    #print ('Experience ', get_experience(document))
    if len(experience) > 1:
        experience_1.append(experience[0])
        experience_2.append(experience[1])
    elif len(experience) == 1:
        experience_1.append(experience[0])
        experience_2.append('')
    elif len(experience) == 0:
        experience_1.append('')
        experience_2.append('')


    if name == None:
      names.append(np.nan)
    else:
      names.append(name)

### To Do
Make the extracted data in a presentable format. Like to an excel file.

In [67]:
import pandas as pd
df = pd.DataFrame({'names':name, 'email id':email_ids, 'phone nos':phone_nos, 'education 1':education_1, 'education 2':education_2, 'skills 1':skills_1,
              'skills 2':skills_2, 'All_skills':skills_ls, 'experience 1':experience_1, 'experiece 2':experience_2})

In [68]:
df.to_excel('extracted_data.xlsx')

In [69]:
!pip install --upgrade scikit-learn

In [72]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer

data = pd.read_excel('extracted_data.xlsx')


data['All_skills'] = data['All_skills'].replace(np.nan,"None")
ls = [z for z in data['All_skills']]

input='html css'
ls.append(input)


#Note: Normally, CV uses analyzer which ingnores C and C++. So, we used tokenizer
tk = TweetTokenizer()
cv = CountVectorizer(tokenizer=tk.tokenize)
count_matrix = cv.fit_transform(ls)
similarity = cosine_similarity(count_matrix)

print(similarity)

[[1.         0.25819889 0.4472136  0.51639778 0.3721042  0.3721042
  0.         0.31622777 0.        ]
 [0.25819889 1.         0.57735027 0.5        0.32025631 0.32025631
  0.         0.         0.81649658]
 [0.4472136  0.57735027 1.         0.28867513 0.         0.
  0.         0.         0.        ]
 [0.51639778 0.5        0.28867513 1.         0.88070485 0.88070485
  0.         0.81649658 0.40824829]
 [0.3721042  0.32025631 0.         0.88070485 1.         1.
  0.         0.78446454 0.39223227]
 [0.3721042  0.32025631 0.         0.88070485 1.         1.
  0.         0.78446454 0.39223227]
 [0.         0.         0.         0.         0.         0.
  1.         0.         0.        ]
 [0.31622777 0.         0.         0.81649658 0.78446454 0.78446454
  0.         1.         0.        ]
 [0.         0.81649658 0.         0.40824829 0.39223227 0.39223227
  0.         0.         1.        ]]


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# print(cv.vocabulary_)
# print(cv.get_feature_names_out())

In [73]:
def n_candidates(input_list,n):

  original_list = input_list.copy()

  unique_list = []
  for num in original_list:
      if num not in unique_list:
          unique_list.append(num)

  sorted_list = sorted(unique_list, reverse=True)

  max_numbers = sorted_list[:n]

  max_indices = []

  for number in max_numbers:
      indices = [i for i, x in enumerate(original_list) if x == number]
      max_indices.extend(indices)

  return max_indices



In [74]:
ls=similarity[:][-1][:-1].tolist()
indices = n_candidates(ls ,4)
print("Indices of the largest numbers:", indices)

Indices of the largest numbers: [1, 3, 4, 5, 0, 2, 6, 7]


In [77]:
for n in indices:
  print(data['names'][n])

abdul kalam
abdul kalam
abdul kalam
abdul kalam
abdul kalam
abdul kalam
abdul kalam
abdul kalam


In [78]:
data

,Unnamed: 0,names,email id,phone nos,education 1,education 2,skills 1,skills 2,All_skills,experience 1,experiece 2
0,0,abdul kalam,vlakhani1010@gmail.com,953 700-2319,data science company,python data science and database management,python,c,python c data science javascript,"['c 05/2022 - 06/2022 +', 'ola electric softwa...","['c 05/2022 - 08/2022 +', 'it based', 'remote'..."
1,1,abdul kalam,apurvbit009@gmail.com,8860651438, btech degree with specialization in computer..., actively participated in sports events like ...,javascript,html,javascript html css,['\uf0b7 previously worked as a software engin...,['\uf0b7 previously worked as a java consultan...
2,2,abdul kalam,ashshakeel0@gmail.com,9170068409,i am a recent graduate of university of kashmi...,bachelor of engineering in computer science,javascript,NaN,javascript,NaN,NaN
3,3,abdul kalam,monika@gmail.com,9039326737,barkatullah university,NaN,c,html,c html css javascript python c++,"['90393267378770212751', 'valejamonika@gmailco...","['duration – aug 2020- present', 'information'..."
4,4,abdul kalam,pharshil2000@gmail.com,9408883467,btech computer science & engineering,nirma university ahmedabad,python,c++,python c++ html css c machine learning,"['may 2018 – may 2022', 'cumulative gpa: 782 /...","['september 2020 – december 2020', '●', 'a web..."
5,5,abdul kalam,NaN,9408883467,btech computer science & engineering,nirma university ahmedabad,python,c++,python c++ html css c machine learning,"['may 2018 – may 2022', 'cumulative gpa: 782 /...","['september 2020 – december 2020', '●', 'a web..."
6,6,abdul kalam,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
7,7,abdul kalam,pagequantum@gmail.com,NaN,of all engineering colleges affiliated to,dr apj abdul kalam technical university,c,c++,c c++,NaN,NaN
